# Red de empresas - Limpieza de los seis años
__Queletzú Paulina Aspra Polo y Mariana Esther Martínez Sánchez__

1. Limpieza por sexenio
* Series de tiempo
* Colaboraciones en contratos
* Contruccion de redes


In [1]:
import pandas as pd
from numpy import nan

from utils import clean_string

import warnings
warnings.filterwarnings("ignore")

In [2]:
def limpiar_df(df,nom):
    """
    Genera tabla de contratos
    """
    print('\traw', df.shape)
    df['monto'] = df['monto'].apply(lambda x: round(x/1000000,3))
    for col in ['direct', 'open', 'limited']:
        df[col] = df['method'].apply(lambda x: 1 if x==col else 0)
        df[col] = df[col] * df['monto']
        df[col] = df[col].replace(0, nan)

    df['consorcio'] = df['empresa']
    df['empresa'] = df['empresa'].apply(lambda x: x.split(';')[0] if type(x)==str else '')
    for col in ['dependencia', 'uc', 'empresa']:
        df[col] = df[col].apply(clean_string)
    df['nombre'] = df[['dependencia', 'uc']].apply(lambda x: x[0]+' ('+x[1]+')' , axis=1)

    df = df[['fecha', 'nombre','uc', 'dependencia', 'empresa', 'consorcio',
             'servicio', 'method', 'monto', 'direct', 'open', 'limited']]
    df.columns = ['fecha', 'nombre','unidad', 'depend', 'empresa', 'consorcio',
                  'servicio', 'metodo', 'monto', 'abierta', 'invtres', 'directa']
    print('\tcontratos', df.shape)
    df.to_csv('data-clean/{}-contratos.csv'.format(nom), index=False)
    return df

def obtener_red(df,nom):
    """
    Genera tabla dependencia-empresa, dependencia y empresa
    """
    
    def clean_groupby(df_):
        df_.columns = df_.columns.droplevel()
        df_ = df_.reset_index().sort_values(['m_inv','c_num'], ascending=False)
        df_[['p_dir', 'p_inv', 'p_abi']] = df_[['p_dir', 'p_inv', 'p_abi']].divide(df_['m_tot'], axis=0)
        return df_

    red = df.groupby(['nombre','empresa']).aggregate({
                      'monto':   {'c_num':'count', 'm_tot':'sum'},
                      'directa': {'c_dir':'count', 'm_dir':'sum', 'p_dir':'sum'},
                      'invtres': {'c_inv':'count', 'm_inv':'sum', 'p_inv':'sum'},
                      'abierta': {'c_abi':'count', 'm_abi':'sum', 'p_abi':'sum'},
                      })
    red = clean_groupby(red)
    print('\tred', red.shape)
    red.to_csv('data-clean/{}-red.csv'.format(nom), index=False)

    for i,j in [["empresa","nombre"], ["nombre","empresa"]]:
        df_ = red.groupby(i) \
                 .aggregate({j:{'count':'count'}, 'c_num':{'c_num':'sum'},
                 'm_tot':{'m_tot':'sum', 'm_mean':'mean', '_max':'max', 'm_min':'min', 'm_std':'std'},
                 'm_dir':{'m_dir':'sum','p_dir':'sum'},
                 'm_inv':{'m_inv':'sum','p_inv':'sum'},
                 'm_abi':{'m_abi':'sum','p_abi':'sum'}
                 })
        df_ = clean_groupby(df_)
        print('\t'+i, df_.shape)
        df_.to_csv('data-clean/{}-{}.csv'.format(nom,i), index=False)

Generar los datos por sexenio

In [3]:
sexenios = {'FOX':[2005,2006], 'CAL':[2011,2012], 'EPN':[2017,2018]}
for pres,años in sexenios.items():
    print(pres)
    df = []
    for año in años:
        df_ = pd.read_csv('data-raw/{}.csv'.format(año))
        df_['fecha'] = año
        print('año',año, df_.shape)
        df.append(df_)
    df = pd.concat(df, ignore_index=True, sort=False)
    df = limpiar_df(df,pres)
    obtener_red(df,pres)

## Para datos anuales:
#for año in ['2005','2006','2011','2012','2017','2018']:
#    print(año)
#    df = pd.read_csv('data-raw/{}.csv'.format(año))   

FOX
año 2005 (59715, 7)
año 2006 (78747, 7)
	raw (138462, 7)
	contratos (138462, 12)
	red (74557, 13)
	empresa (44625, 14)
	nombre (2660, 14)
CAL
año 2011 (274101, 7)
año 2012 (275061, 7)
	raw (549162, 7)
	contratos (549162, 12)
	red (122254, 13)
	empresa (76041, 14)
	nombre (4295, 14)
EPN
año 2017 (272353, 7)
año 2018 (180016, 7)
	raw (452369, 7)
	contratos (452369, 12)
	red (172559, 13)
	empresa (100765, 14)
	nombre (4470, 14)
